In [1]:
'''
LOAD IMPORTANT PUZZLES
'''

import numpy as np
import openpyxl
import csv

importantPuzzles = []
allThemes = set()
with open('ImportantPuzzles.csv', newline='') as csvfile:
    first_row = True
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"', escapechar='\\')
    for row in spamreader:
        if not first_row:
            themes = [theme.strip(" '") for theme in row[10].strip("[]").split(',')]
            row.append(themes)
            allThemes.update(themes)
        else:
            first_row = False
        importantPuzzles.append(row)

#row[0] = PuzzleID
#row[1] = FEN
#row[2] = Moves
#row[3] = Rating
#row[4] = RatingDeviation
#row[5] = Popularity
#row[6] = NbPlayes
#row[7] = Themes
#row[8] = URL
#row[9] = OpeningTags
#row[10] = GoodRowThemes
#row[11] = CountGoodRowThemes
#row[12] = GoodRowThemes but Formatted into List
header = importantPuzzles[0]
header.append("GoodRowThemesFormatted")
importantPuzzles.remove(header)

In [2]:
'''
GET COMPLETED PUZZLES SOLVED DURING INITIAL AND TEST PHASE
'''

completedPuzzles = []
completedIDs = []

csvFilenames = ['Participant1_1300/Participant1_TestPuzzles.csv', 'Participant1_1300/Participant1_InitialPuzzles_1300.csv'] #this can be 

for i in range(len(csvFilenames)):
    donePuzzles = []
    with open(csvFilenames[i], newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in spamreader:
            donePuzzles.append(row)
    donePuzzles.remove(donePuzzles[0])

    for puzzle in donePuzzles[1:]:
        completedPuzzles.append(puzzle)
        completedIDs.append(puzzle[0])

Now we need to generate our final puzzles that are used for comparisons.

To generate these puzzles we select puzzles that the player has not played during the initial puzzles or training phase.

The puzzles have the same requirements as the initial puzzles, being in a rating range of +-80 of the inital rating. Also every theme has to occur at least 4 times.

This time we used a more efficient algorithm for our puzzle selection, randomizing the entire list and iterating through that, that way we don't check puzzles twice.
We also update the needed amount for each skill after adding a puzzle without having to iterate through our final puzzles again.

The only difference is the ratingIncrement. Because we don't have enough puzzles in a +-80 rating difference for some themes (in our case "enPassant" for Participant1) we need to update the rating difference if we don't find enough puzzles in the +-80 range.

In [7]:
import pandas as pd
import os
import random

#This adds all possible puzzles that we can choose from
def getPossiblePuzzles(rating, importantPuzzles, completedIDs, importantThemes, timesToOccur, ratingIncrement=80, maxRatingDifference=400):
    posPuzz = []
    specificThemePuzzles = {}
    theme_count = {theme: 0 for theme in importantThemes}

    # Initial search within the rating increment
    for puzzle in importantPuzzles:
        if (rating - ratingIncrement < int(puzzle[3]) < rating + ratingIncrement and
                puzzle[0] not in completedIDs): #Check if the puzzle is in the rating range and has not been played
            posPuzz.append(puzzle)
            for theme in puzzle[12]:
                if theme in importantThemes:
                    theme_count[theme] += 1

    # Identify themes that need more occurrences
    themes_needing_more = [theme for theme, count in theme_count.items() if count < timesToOccur]

    # Expand search for specific themes that didn't meet the requirement
    currentRatingDifference = ratingIncrement
    while themes_needing_more and currentRatingDifference <= maxRatingDifference:
        for puzzle in importantPuzzles:
            if (rating < int(puzzle[3]) < rating + currentRatingDifference and
                    puzzle[0] not in completedIDs and
                    any(theme in puzzle[12] for theme in themes_needing_more)):
                for theme in puzzle[12]:
                    if theme in themes_needing_more:
                        if theme not in specificThemePuzzles:
                            specificThemePuzzles[theme] = []
                        specificThemePuzzles[theme].append(puzzle)
                        theme_count[theme] += 1
                        if theme_count[theme] >= timesToOccur:
                            themes_needing_more.remove(theme)

        currentRatingDifference += ratingIncrement #Increasing the rating range

    return posPuzz, specificThemePuzzles, theme_count

#This checks for the 'timesToOccur' requirement
def getPuzzleListToOccurMinTimes(generalPuzzles, specificThemePuzzles, timesToOccur, themesToCheck, theme_count_initial):

    selectedPuzzles = []
    theme_count = {theme: 0 for theme in themesToCheck}

    # Handle specific themes first, as these require a different method
    for theme, puzzles in specificThemePuzzles.items():
        timesNeeded = timesToOccur - theme_count_initial.get(theme,0)
        sortedPuzzles = sorted(puzzles, key=lambda x: int(x[3])) #sorting by rating
        for puzzle in sortedPuzzles:
            if theme_count[theme] < timesNeeded:
                selectedPuzzles.append(puzzle)
                theme_count[theme] += 1

    # Create a shuffled list of indices for generalPuzzles
    indices = list(range(len(generalPuzzles)))
    random.shuffle(indices)

    for index in indices:
        if all(count >= timesToOccur for count in theme_count.values()):
            break

        puzzle = generalPuzzles[index]
        themes = puzzle[12]

        if any(theme_count.get(theme, 0) < timesToOccur for theme in themes):
            selectedPuzzles.append(puzzle)
            for theme in themes:
                theme_count[theme] = theme_count.get(theme, 0) + 1

    return selectedPuzzles

candidateRating = 1300 #Rating for the participant we are generating the puzzles for
timesToOccur = 4 #Every theme has to occur at least 4 times

# Getting possible puzzles and specific theme puzzles
possiblePuzzles, specificThemePuzzles, theme_count_initial = getPossiblePuzzles(candidateRating, importantPuzzles, completedIDs, allThemes, timesToOccur)

# Generating the final puzzle list
finalPuzzleList = getPuzzleListToOccurMinTimes(possiblePuzzles, specificThemePuzzles, timesToOccur, allThemes, theme_count_initial)

#Shuffling the final list again
finalIndices = list(range(len(finalPuzzleList)))
random.shuffle(finalIndices)
shuffledFinalPuzzleList = [finalPuzzleList[i] for i in finalIndices]


'''
PRINT FILES
'''

candidateName = csvFilenames[0].split('/')[0]

if not os.path.exists(candidateName):
    os.makedirs(candidateName)

xlsxFilename = candidateName + '/' + candidateName + "_FinalPuzzles.xlsx"
csvFilename = candidateName + '/' + candidateName + "_FinalPuzzles.csv"

finalIDs = []
for puzzle in shuffledFinalPuzzleList:
    finalIDs.append (["=HYPERLINK(\"https://lichess.org/training/" + puzzle[0] + "\")", ""])

#Get New CSV File with all initial Puzzles
xlsxHeader = ["PuzzleLink","Richtig(1) oder Falsch(0)"]
finalIDs.insert(0, xlsxHeader)

shuffledFinalPuzzleList.insert(0, header)

with open(csvFilename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(shuffledFinalPuzzleList)

df = pd.DataFrame(finalIDs[1:], columns=finalIDs[0])
df.to_excel(xlsxFilename, index=False)

print(len(shuffledFinalPuzzleList))

88
